In [1]:
import requests
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import csv

In [2]:
# last page count
page_start=4
group_range=(page_start*30)-30
page_stop=15
itera=1

for pg in range(page_start,page_stop):
    url=f"https://www.magicbricks.com/mbsrp/propertySearch.html?editSearch=Y&category=R&propertyType=10002,10003,10021,10022,10020,10001,10017&city=3649&page={page_start}&groupstart={group_range}&offset=0&maxOffset=22&sortBy=premiumRecent&postedSince=-1&pType=10002,10003,10021,10022,10020,10001,10017&isNRI=N&multiLang=en"
    r=requests.get(url)
    data=r.json()
    a=[]
    for result in data["resultList"]:
        a.append(result["id"])
    t1=a[0]
    
    url_1=f"https://www.magicbricks.com/mbsrp/propertySearch.html?editSearch=Y&category=R&propertyType=10002,10003,10021,10022,10020,10001,10017&city=3649&page={page_start+1}&groupstart={group_range+30}&offset=0&maxOffset=22&sortBy=premiumRecent&postedSince=-1&pType=10002,10003,10021,10022,10020,10001,10017&isNRI=N&multiLang=en"
    r=requests.get(url_1)
    data=r.json()
    b=[]
    for result in data["resultList"]:
        b.append(result["id"])
    t2=b[0]
    if t1==t2:
        print("got it")
        print(t1,t2)
        print(page_start)
        page_end=page_start
        print("page_end: ",page_end)
        break
    else:
        page_start=page_start+1
        group_range=group_range+30
        print(itera,"page",page_start, "Id-",t1)
        itera=itera+1
        

1 page 5 Id- 42612085
2 page 6 Id- 42169317
got it
64877563 64877563
6
page_end:  6


In [3]:
mgb = {"unique_id":[],"City":[],"Rent_price":[],"CarpetArea":[],
       "CarpetUnit":[],"CoveredArea":[],"CoveredUnit":[],"latitude":[],"longitude":[]
       ,"Post_date":[],"Property_Name":[],"Property_Address":[],"url":[]}
page_start=1
group_range=(page_start*30)-30
page_stop=page_end+1

for pg in range(page_start,page_stop):
    url=f"https://www.magicbricks.com/mbsrp/propertySearch.html?editSearch=Y&category=R&propertyType=10002,10003,10021,10022,10020,10001,10017&city=3649&page={page_start}&groupstart={group_range}&offset=0&maxOffset=22&sortBy=premiumRecent&postedSince=-1&pType=10002,10003,10021,10022,10020,10001,10017&isNRI=N&multiLang=en"
    r=requests.get(url)
    data=r.json()
    for result in data["resultList"]:
        try:
            mgb["unique_id"].append(result["id"])
        except:
            mgb["unique_id"].append(np.nan)
         
        try:
            corr=result["ltcoordGeo"].split(",")
            mgb["latitude"].append(corr[0])
            mgb["longitude"].append(corr[1])
        except:
            mgb["latitude"].append(np.nan)
            mgb["longitude"].append(np.nan)
            
        try:
            mgb["Rent_price"].append(result["price"])
        except:
            mgb["Rent_price"].append(np.nan) 
            
        try:
            mgb["CarpetArea"].append(result["carpetArea"])
            mgb["CarpetUnit"].append("Sq-ft")
        except:
            mgb["CarpetArea"].append(np.nan)
            mgb["CarpetUnit"].append(np.nan)
            
        try:
            mgb["CoveredArea"].append(result["coveredArea"])
            mgb["CoveredUnit"].append("Sq-ft")
        except:
            mgb["CoveredArea"].append(np.nan)
            mgb["CoveredUnit"].append(np.nan)
        try:
            mgb["City"].append(result["ctName"])
        except:
            mgb["City"].append(np.nan)
            
        
        try:
            mgb["Property_Name"].append(result["propertyTitle"])
        except:
             mgb["Property_Name"].append(np.nan)
                
        try:
            fetch_url = result["url"]
            url_link = f"https://www.magicbricks.com/propertyDetails/{fetch_url}"
            mgb["url"].append(url_link)
        except:
             mgb["url"].append(np.nan)

        try:
            r = requests.get(url_link).text
            soup = BeautifulSoup(r, "lxml")
            for i in range(0,5):
                title=soup.find_all("div", class_="mb-ldp__more-dtl__list--label")[i].text
                if title=="Address":
                    address = soup.find_all("div", class_="mb-ldp__more-dtl__list--value")[i].text
                    mgb["Property_Address"].append(address)
                    break
                else:
                   continue
        except:
            mgb["Property_Address"].append(np.nan)
            
        try:
            mgb["Post_date"].append(result["postDateT"][0:10])
        except:
            mgb["Post_date"].append(np.nan)
    print("Page complete -- ",page_start)
    page_start=page_start+1    
    group_range=group_range+30
mgb_f=pd.DataFrame(mgb)
print("All done")

Page complete --  1
Page complete --  2
Page complete --  3
Page complete --  4
Page complete --  5
Page complete --  6
All done


In [20]:
mgb_f=pd.DataFrame(mgb)

In [21]:
mgb_f.shape

(180, 13)

In [22]:
mgb_uni=mgb_f[mgb_f["unique_id"].duplicated()==False]

In [23]:
mgb_uni.shape

(161, 13)

In [24]:
mgb_uni.to_csv("‪magic_bricks_Kozhikode.csv")